In [68]:
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf
import os
import pickle

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

### Fixing keyerror

In [69]:
from music21 import converter, corpus, instrument, midi, note, chord, pitch


"""path='../raw_data/music_21/'

def open_midi(midi_path):
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()
    return midi.translate.midiFileToStream(mf)
    
base_midi = open_midi(concat_path(path, "test_file.mid"), True)
base_midi"""

data_path = pathlib.Path('../raw_data/raw_files/chopin_de_format_0/')
filenames = glob.glob(str(data_path/"*.mid"))
print('Number of files:', len(filenames))


Number of files: 38


In [125]:
score = converter.parse(filenames[3])

In [124]:
print(filenames[3])

../raw_data/raw_files/chopin_de_format_0/chpn-p19_format0.mid


In [126]:
score

<music21.stream.Score 0x29bb0e410>

In [31]:
s2 = instrument.partitionByInstrument(score)

In [37]:
s2

<music21.stream.Score 0x2a351edd0>

In [34]:
def list_instruments(midi):
    partStream = midi.parts.stream()
    print("List of instruments found on MIDI file:")
    for p in partStream:
        aux = p
        print (p.partName)


list_instruments(score)

List of instruments found on MIDI file:
Chopin Prelude  No. 5, Opus 28


In [127]:
components = []
for element in score.recurse():
    components.append(element)

In [128]:
len(components)

1609

In [129]:
components

 <music21.stream.Part 0x29d3c1030>,
 <music21.stream.Measure 1 offset=0.0>,
 <music21.instrument.Piano 'Chopin Prelude  No. 19, Opus 28: Piano'>,
 <music21.instrument.Piano 'Chopin Prelude  No. 19, Opus 28: Piano'>,
 <music21.clef.TrebleClef>,
 <music21.tempo.MetronomeMark Quarter=204.09>,
 <music21.key.Key of E- major>,
 <music21.meter.TimeSignature 3/4>,
 <music21.note.Rest half>,
 <music21.tempo.MetronomeMark Quarter=186.29>,
 <music21.note.Note B->,
 <music21.stream.Measure 2 offset=3.0>,
 <music21.tempo.MetronomeMark Quarter=194.76>,
 <music21.tempo.MetronomeMark Quarter=190.0>,
 <music21.chord.Chord B-4 E-2>,
 <music21.tempo.MetronomeMark Quarter=190.53>,
 <music21.chord.Chord B-3 G3>,
 <music21.tempo.MetronomeMark Quarter=191.06>,
 <music21.chord.Chord G4 B-2>,
 <music21.tempo.MetronomeMark Quarter=191.59>,
 <music21.chord.Chord E-5 E-2>,
 <music21.chord.Chord E-4 G3>,
 <music21.tempo.MetronomeMark Quarter=194.23>,
 <music21.chord.Chord B-4 B-2>,
 <music21.tempo.MetronomeMark Qu

### collapse


In [62]:
notes = []
for element in components:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch) + " " +  str(element.quarterLength))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
    elif isinstance(element, note.Rest):
        notes.append(str(element.name)  + " " + str(element.quarterLength))

In [63]:
notes

['B4 0.25',
 'A4 0.25',
 '1.7 0.25',
 '9.11 0.25',
 '4.7 0.25',
 '4.9 0.25',
 '1.7 0.25',
 '9.10 0.25',
 '4.7 0.25',
 '4.9 0.25',
 '1.7 0.25',
 '9.11 0.25',
 '4.7 0.25',
 '4.9 0.25',
 '1.7 0.25',
 '9.10 0.25',
 '4.7 0.25',
 '4.9 0.25',
 '9.1 0.25',
 '9.11 0.25',
 '4.7 0.25',
 '4.9 0.25',
 '1.7 0.25',
 '6.9 0.25',
 '2 0.25',
 '6.9 0.25',
 '7.10 0.25',
 '7.11 0.25',
 '6.9.1 0.25',
 '4.7 0.25',
 '2.6 0.25',
 '11.2 0.25',
 '2.6 0.25',
 '6.9 0.25',
 '1.4.8 0.25',
 '11.2 0.25',
 '1.4 0.25',
 '9 0.25',
 '2.5 0.25',
 '2.6 0.25',
 '1.4.8 0.25',
 '11.2 0.25',
 '9.1 0.25',
 '0.4 0.25',
 '4.6 0.25',
 '9 0.25',
 '11.0.3 0.25',
 '9.11 0.25',
 '7.11 0.25',
 '4 0.25',
 '9.0 0.25',
 '4.9 0.25',
 '3.7.11 0.25',
 '6.9 0.25',
 '4.7 0.25',
 '4 0.25',
 '4.7 0.25',
 '7.11 0.25',
 '2.6.10 0.25',
 '1.4 0.25',
 '11.2 0.25',
 '6.11 0.25',
 '4.7 0.25',
 '11.4 0.25',
 '2.6.10 0.25',
 '1.4 0.25',
 '11.2 0.25',
 '5.11 0.25',
 '8.11 0.25',
 '2.8 0.25',
 '1 0.25',
 '11.2 0.25',
 '10.1 0.25',
 '6 0.25',
 '11.2 0.25',
 

In [ ]:
"""for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(element.quarterLength))

    with open('..raw_data/music_21', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes"""

In [85]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./full_set_beethoven_mozart directory. Call BEFORE train """
    notes = []
    durations = []

    for file in glob.glob("../raw_data/raw_files/chopin_de_format_0/*.mid"):
        try:
            score = converter.parseFile(file)
        except:
            return file
        """print("Parsing %s" % file)
        notes_to_parse = []
        for element in score.recurse():
            notes_to_parse.append(element)
        print("OK")"""
    


In [86]:
notes = get_notes()

In [90]:
print(notes)
converter.parseFile(notes)

../raw_data/raw_files/chopin_de_format_0/chpn_op35_2_format0.mid


KeyError: 11859736560

In [35]:
notes

NameError: name 'notes' is not defined

In [74]:

def get_notes():
    """ Get all the notes and chords from the midi files. Call BEFORE train """
    notes = []
    #durations = []

    for file in glob.glob("../raw_data/music_21/*.mid"):
        midi = converter.parse(file)
        print("Parsing %s" % file)
        #notes_to_parse = None

        components = []
        for element in midi.recurse():
            components.append(element)

        for element in components:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(element.quarterLength))

    return notes


In [75]:
get_notes()

Parsing ../raw_data/music_21/chpn_op10_e01_format0.mid
Parsing ../raw_data/music_21/chpn_op7_1_format0.mid
Parsing ../raw_data/music_21/chpn_op7_2_format0.mid


['C2 4.0',
 'C3 0.25',
 'C3 3.75',
 'rest 0.5',
 'G3 0.25',
 'C4 0.25',
 'E4 0.25',
 'C4 0.25',
 'G4 0.25',
 'C5 0.25',
 'E5 0.25',
 'C5 0.25',
 'G5 0.25',
 'C6 0.25',
 'E6 0.25',
 'C6 0.25',
 'G6 0.25',
 'C7 0.25',
 'E7 0.25',
 'C2 4.0',
 'C3 4.0',
 'C7 0.25',
 'G6 0.25',
 'C6 0.25',
 'E6 0.25',
 'C6 0.25',
 'G5 0.25',
 'C5 0.25',
 'E5 0.25',
 'C5 0.25',
 'G4 0.25',
 'C4 0.25',
 'E4 0.25',
 'C4 0.25',
 'G3 0.25',
 'C3 0.25',
 '5 4.0',
 'rest 0.25',
 'C3 0.25',
 'A3 0.25',
 'C4 0.25',
 'F4 0.25',
 'C4 0.25',
 'A4 0.25',
 'C5 0.25',
 'F5 0.25',
 'C5 0.25',
 'A5 0.25',
 'C6 0.25',
 'F6 0.25',
 'C6 0.25',
 'A6 0.25',
 'C7 0.25',
 'E7 0.25',
 'C7 0.25',
 'A6 0.25',
 'C6 0.25',
 'E6 0.25',
 'C6 0.25',
 'A5 0.25',
 'C5 0.25',
 'E5 0.25',
 'C5 0.25',
 'A4 0.25',
 'C4 0.25',
 'D4 0.25',
 'C4 0.25',
 'A3 0.25',
 'C3 0.25',
 '6 4.0',
 '7 2.0',
 'D5 0.25',
 'B4 0.25',
 'G5 0.25',
 'B5 0.25',
 'D6 0.25',
 'B5 0.25',
 'G6 0.25',
 'A6 0.25',
 'rest 0.25',
 'B2 0.25',
 'G3 0.25',
 'B3 0.25',
 'D4 0.2

In [94]:
def get_notes(filename):
    buffer_size = 500000
    notes = [''] * buffer_size
    last_time = 0
    offset = []
    midi = converter.parse(filename)
    print('parsing {}'.format(filename))
    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            time = int(element.offset * 4)
            if notes[time] == '':
                notes[time] = str(element.pitch.midi)
            last_time = time
        elif isinstance(element, chord.Chord):
            time = int(element.offset * 4)
            if notes[time] == '':
                notes[time] = '.'.join(str(n) for n in element.normalOrder) 
            last_time = time
    print(last_time)
    return notes[:last_time]

In [78]:
get_notes('../raw_data/midi_chopin_format_0_dataset/chpn-p1_format0.mid')

parsing ../raw_data/midi_chopin_format_0_dataset/chpn-p1_format0.mid
135


['7.0',
 '55',
 '0.4.7',
 '9.11',
 '',
 '55',
 '2.5.7',
 '9.0',
 '',
 '55',
 '0.4.7',
 '4.9',
 '',
 '59',
 '4.7.11',
 '0.5',
 '',
 '64',
 '9.0.4',
 '2.6',
 '',
 '64',
 '9.0.4',
 '2.7',
 '',
 '64',
 '4.7.11',
 '2.7',
 '',
 '59',
 '11.2.5',
 '9.0',
 '',
 '55',
 '0.4.7',
 '9.11',
 '',
 '55',
 '2.5.7',
 '9.0',
 '',
 '55',
 '0.4.7',
 '4.9',
 '',
 '59',
 '4.7.11',
 '0.5',
 '',
 '61',
 '1.5.9',
 '2.7',
 '',
 '63',
 '0.3.7',
 '4.9',
 '',
 '67',
 '7.9.0',
 '5.7',
 '',
 '63',
 '0.3.7',
 '4.9',
 '',
 '64',
 '9.0.4',
 '5.11',
 '66',
 '7.11.2',
 '11.2.6',
 '7.0',
 '68',
 '0.4.7',
 '0.4.8',
 '9.2',
 '70',
 '2.5.7',
 '7.10.2',
 '11.4',
 '',
 '74',
 '2.4.7',
 '0.6',
 '',
 '71',
 '11.0.3',
 '7.9',
 '69',
 '0.4',
 '9.0.4',
 '7',
 '',
 '64',
 '4.7.11',
 '0.2',
 '55',
 '0.4.7',
 '0.4.7',
 '9.0',
 '64',
 '5.7',
 '4.7.11',
 '0.2',
 '',
 '55',
 '0.4.7',
 '9.0',
 '',
 '64',
 '4.7.11',
 '0.2',
 '',
 '5.7.0',
 '72',
 '0.4.7',
 '72',
 '5.7.0',
 '72',
 '0.4.7',
 '72',
 '5.7.0',
 '72',
 '0.4.7',
 '72',
 '5.7.0',
 

In [79]:
def prepare_sequences(notes, n_vocab, note_to_int):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 400
    i  = []
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)
    network_output = to_categorical(network_output)
    return (network_input, normalized_input, network_output)

def normalize(network_input, n_vocab):
    sequence_length = 400
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)
    return normalized_input

In [81]:
file_names = glob.glob("../raw_data/raw_files/chopin_de_format_0/*.mid")
print(file_names)
notes = []
for song in file_names:
    notes = notes + get_notes(song)

['../raw_data/raw_files/chopin_de_format_0/chpn-p15_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn_op25_e12_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p5_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p19_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn_op10_e12_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p9_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p10_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn_op35_4_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p22_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chp_op18_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p21_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p13_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn_op25_e1_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p3_format0.mid', '../raw_data/raw_files/chopin_de_format_0/chpn-p6_format0.mid', '../raw_dat

KeyError: 11873775968

In [84]:
data_path = '../raw_data/midi_chopin_format_0_dataset/'
for file in glob.glob("../raw_data/midi_chopin_format_0_dataset/*.mid"):
    print(get_notes(file))

parsing ../raw_data/midi_chopin_format_0_dataset/chpn-p15_format0.mid
1416
['1.5', '', '56', '73', '1.5.8', '', '56', '', '56', '', '56', '', '70', '', '56', '', '0.3.6', '', '56', '', '56', '', '56', '', '58', '', '56', '', '73', '', '56', '', '3.6', '', '56', '5.8', '6.10', '', '56', '', '56', '', '56', '', '8.1', '', '56', '', '68', '', '56', '', '66', '', '3.8', '', '73', '', '56', '3', '0.5.6', '2.3', '5.8', '78', '1.5', '', '56', '73', '68', '', '56', '', '56', '', '56', '', '70', '', '56', '', '0.3.6', '', '56', '', '56', '', '56', '', '58', '', '56', '', '73', '', '56', '', '3.6', '', '56', '5.8', '6.10', '', '56', '', '56', '', '56', '', '8.1', '', '56', '', '68', '', '56', '', '66', '', '3.8', '', '73', '', '56', '', '0.1.5', '', '8.1', '', '75', '', '56', '', '75', '', '56', '', '11.3', '', '56', '', '77', '', '61', '', '6.10', '', '1.5', '', '3.6.10', '', '1', '', '77', '', '56', '', '1.5', '', '56', '', '3', '', '8.11', '', '10.11.3', '', '8', '', '70', '', '55', '8.10.11'

KeyError: 11265926016

In [107]:
file_names = glob.glob("../raw_data/midi_chopin_format_0_dataset/*.mid")
print(file_names)
notes = []
file_with_error=[]
for song in file_names:
    try:
        notes = notes + get_notes(song)
    except KeyError:
        file_with_error.append(song)

['../raw_data/midi_chopin_format_0_dataset/chpn-p15_format0.mid', '../raw_data/midi_chopin_format_0_dataset/chpn_op25_e12_format0.mid', '../raw_data/midi_chopin_format_0_dataset/sonata_58_4_(c)finley.mid', '../raw_data/midi_chopin_format_0_dataset/etude_nouvelles_1_(c)unknown.mid', '../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_63_1_(c)lubetsky.mid', '../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_59_2_(c)lubetsky.mid', '../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_24_4_(c)lubetsky.mid', '../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_17_2_(c)lubetsky.mid', '../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_7_4_(c)lubetsky.mid', '../raw_data/midi_chopin_format_0_dataset/64833a_nocturne_op_32_no_1_b_major_(nc)smythe.mid', '../raw_data/midi_chopin_format_0_dataset/52645a_valse_brillante_op_34_no_1_a_flat_(nc)smythe.mid', '../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_68_2_(c)lubetsky.mid', '../raw_data/midi_chopin_format_0_data

In [108]:
len(file_with_error)

0

In [109]:
file_with_error

[]

In [110]:
len(notes)

218004

### Testing generating music LSTM model with attention (Music21)

In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import glob
import pickle

from music21 import converter, instrument, stream, note, chord

#Run version 2.1.6
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

In [130]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./full_set_beethoven_mozart directory. Call BEFORE train """
    notes = []
    durations = []

    for file in glob.glob("../raw_data/midi_chopin_format_0_dataset/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi) #Change to only grab the piano???
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(float(element.quarterLength)))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(float(element.quarterLength)))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(float(element.quarterLength)))

    return notes

In [135]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 20

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = utils.to_categorical(network_output)

    return (network_input, network_output)


In [136]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


In [137]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = os.path.abspath("weights-1LSTMAtt1LSTMLayer-{epoch:03d}-{loss:.4f}.hdf5")
    checkpoint = ModelCheckpoint(
        filepath,
        period=10, #Every 10 epochs
        monitor='loss',
        verbose=1,
        save_best_only=False,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)


In [138]:
def train_network(notes, n_vocab):
    """ Train a Neural Network to generate music """
    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)


In [140]:
#load files in
notes = get_notes()

# get amount of pitch names
n_vocab = len(set(notes))


Parsing ../raw_data/midi_chopin_format_0_dataset/chpn-p15_format0.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/chpn_op25_e12_format0.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/sonata_58_4_(c)finley.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/etude_nouvelles_1_(c)unknown.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_63_1_(c)lubetsky.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_59_2_(c)lubetsky.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_24_4_(c)lubetsky.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_17_2_(c)lubetsky.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_7_4_(c)lubetsky.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/64833a_nocturne_op_32_no_1_b_major_(nc)smythe.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/52645a_valse_brillante_op_34_no_1_a_flat_(nc)smythe.mid
Parsing ../raw_data/midi_chopin_format_0_dataset/chopin_mazurkas_

In [141]:
n_vocab

7496

In [142]:
#train
train_network(notes, n_vocab) 

Epoch 1/200


/Users/aurelie_emma/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2022-12-08 13:32:06.940372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-08 13:32:07.408550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-08 13:32:07.429276: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-08 13:32:07.820527: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registr

   1/3466 [..............................] - ETA: 4:31:09 - loss: 0.0000e+00

2022-12-08 13:32:08.285424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-08 13:32:08.316265: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3466/3466 [==============================] - 847s 243ms/step - loss: 0.0000e+00
Epoch 2/200
3466/3466 [==============================] - 841s 243ms/step - loss: 0.0000e+00
Epoch 3/200
3466/3466 [==============================] - 844s 244ms/step - loss: 0.0000e+00
Epoch 4/200
3466/3466 [==============================] - 873s 252ms/step - loss: 0.0000e+00
Epoch 5/200
 203/3466 [>.............................] - ETA: 13:50 - loss: 0.0000e+00

KeyboardInterrupt: 